<a href="https://colab.research.google.com/github/tomasonjo/blogs/blob/master/neo4jdocs/Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install selenium==4.7.2 transformers sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!apt-get update
!apt install chromium-chromedriver
!apt install -y xvfb

!pip install undetected-chromedriver
!pip install PyVirtualDisplay

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [1,073 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:1

In [3]:
!zip -j /content/chromedriver_linux64.zip /usr/bin/chromedriver
#replace python3.8 with your own version of python in case it's not the same
patcher_src = "/usr/local/lib/python3.8/dist-packages/undetected_chromedriver/patcher.py"
with open(patcher_src, "r") as f:
    contents = f.read()
    contents = contents.replace("return urlretrieve(u)[0]",\
                     "return urlretrieve('file:///content/chromedriver_linux64.zip',"\
                     "filename='/tmp/chromedriver_linux64.zip')[0]")
with open(patcher_src, "w") as f:
    f.write(contents)

  adding: chromedriver (deflated 51%)


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import undetected_chromedriver.v2 as uc
from pyvirtualdisplay import Display
display = Display(visible=0, size=(800, 600))
display.start()

options = uc.ChromeOptions()
options.add_argument("--no-sandbox")
wd = uc.Chrome(options=options)

In [6]:
def extract_text_by_class(c):
  global wd
  try:
    content = wd.find_element(By.CLASS_NAME, c)
    return content.text
  except:
    return ""

def extract_links_by_xpath(xpath):
  global wd
  links = set()
  try:
    a_elems = wd.find_elements(By.XPATH, xpath)
    for elem in a_elems:
        link = elem.get_attribute("href")
        if link == "javascript:void(0)":
          continue
        # Remove links to images and various files
        if link.endswith('.png') or link.endswith('.json') or link.endswith('.txt') or link.endswith('.svg') or \
             link.endswith('.ipynb') or link.endswith('.jpg') or link.endswith('.pdf') or link.endswith('.mp4'):
          continue
        # Remove anchors
        link = link.split("#")[0]
        # Remove parameters
        link = link.split("?")[0]
        # Remove trailing forward slash
        link = link.rstrip("/")
        links.add(link)
    return list(links)
  except:
    return []



In [7]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, AutoModel
from transformers import pipeline
import torch

tokenizer = AutoTokenizer.from_pretrained("yanekyuk/bert-uncased-keyword-extractor")
model = AutoModelForTokenClassification.from_pretrained("yanekyuk/bert-uncased-keyword-extractor")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)

def extract_keywords(text):
  """
  Extract keywords and construct them back from tokens
  """
  result = list()
  keyword = ""
  ner_results = nlp(text)
  a = [x["word"] for x in ner_results]
  for token in a:
    if token.startswith("##"):
      keyword += token[2:]
    else:
      if keyword:
        result.append(keyword)
      keyword = token
  return result


In [8]:
extract_keywords("""
Web APIs are a huge opportunity to access and integrate data from any sources with your graph. Most of them provide the data in JSON format.

The Load JSON procedures retrieve data from URLs or maps and turn it into map value(s) for Cypher to consume. Cypher has support for deconstructing nested documents with dot syntax, slices, UNWIND etc. so it is easy to turn nested data into graphs.

Sources with multiple JSON objects (JSONL,JSON Lines) in a stream, like the streaming Twitter format or the Yelp Kaggle dataset, are also supported,
""")

['graph', 'json', 'cypher', 'syntax', 'twitter', 'yelp', 'kaggle']

In [9]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

def generate_embeddings(text):
  embeddings = model.encode(text)
  return embeddings

In [10]:
generate_embeddings("""
Web APIs are a huge opportunity to access and integrate data from any sources with your graph. Most of them provide the data in JSON format.

The Load JSON procedures retrieve data from URLs or maps and turn it into map value(s) for Cypher to consume. Cypher has support for deconstructing nested documents with dot syntax, slices, UNWIND etc. so it is easy to turn nested data into graphs.

Sources with multiple JSON objects (JSONL,JSON Lines) in a stream, like the streaming Twitter format or the Yelp Kaggle dataset, are also supported,
""")

array([-6.91139773e-02,  5.22450246e-02, -7.20175402e-03, -1.53747583e-02,
        3.90605256e-03, -5.04245870e-02, -1.06758304e-01,  6.22570850e-02,
        3.25472723e-03,  1.30456015e-02,  7.43217533e-03,  2.43865862e-03,
        7.32424557e-02,  6.01712912e-02,  1.34542882e-01,  3.66639942e-02,
       -7.08860531e-03,  3.01807616e-02,  2.62598991e-02, -1.47831529e-01,
        1.14754327e-02,  1.68466021e-03, -4.65278551e-02, -3.76555175e-02,
        4.88043465e-02,  8.55944306e-02, -3.02659739e-02,  5.21276854e-02,
        4.56754006e-02,  6.37385398e-02, -2.97765303e-02, -1.06713623e-02,
       -5.80438562e-02,  5.81364110e-02, -9.82052982e-02,  9.66788828e-03,
        3.53001803e-02,  9.53009259e-03,  1.33905048e-02,  5.26607707e-02,
        4.21146117e-02,  1.21976109e-02, -4.74480353e-02,  1.82718523e-02,
       -2.74318503e-03, -1.94642749e-02, -8.98622945e-02,  4.01607007e-02,
       -5.03845923e-02,  5.59670553e-02, -1.13629147e-01, -4.88040633e-02,
        9.09030996e-03,  

In [ ]:
from selenium.webdriver.common.by import By

entry_url = "https://neo4j.com/docs"
data = dict()
visit_list = [entry_url]
already_visited = []

while visit_list:
  # Visit the URL
  current_url = visit_list.pop()
  if current_url in already_visited:
    continue
  print(current_url)
  try:
    wd.get(current_url)
  except:
    print(f"Couldn't open {current_url}")
  # Extract text from the content div  
  text = extract_text_by_class("content")
  # If nothing is found, try article div
  if not text:
    text = extract_text_by_class("article")

  # Generate paragraph embedding & extract keywords
  if text:
    embeddings = generate_embeddings(text)
    keywords = extract_keywords(text)
  else:
    embeddings = []
    keywords = []

  # Extract links from the content div
  links = extract_links_by_xpath("//div[@class='content']//a[@href]")
  # If nothing is found, try article div
  if not links:
    links = extract_links_by_xpath("//article[@class='article']//a[@href]")
  
  if not links and not text:
    print(f"Couldn't retrieve the data from {current_url}")
  # Store page information
  data[current_url] = {'links': links, 'text': text, 'embeddings': embeddings, 'keywords': keywords}
  # Crawling information
  already_visited.append(current_url)
  # Don't leave neo4j.com while crawling
  visit_list.extend([l for l in list(links) if "neo4j.com" in l and not l in already_visited])

Streaming output truncated to the last 5000 lines.
https://neo4j.com/labs/apoc/4.4/overview/apoc.systemdb/apoc.systemdb.graph
https://neo4j.com/labs/apoc/4.4/database-introspection/config
https://neo4j.com/labs/apoc/4.4/overview/apoc.config/apoc.config.list
https://neo4j.com/labs/apoc/4.4/overview/apoc.config/apoc.config.map
https://neo4j.com/labs/apoc/4.4/database-introspection/meta
https://neo4j.com/labs/apoc/4.4/overview/apoc.meta/apoc.meta.graph.of
https://neo4j.com/labs/apoc/4.4/overview/apoc.meta/apoc.meta.graph
https://neo4j.com/labs/apoc/4.4/overview/apoc.meta/apoc.meta.relTypeProperties
https://neo4j.com/labs/apoc/4.4/overview/apoc.meta/apoc.meta.cypher.types
https://neo4j.com/labs/apoc/4.4/overview/apoc.meta/apoc.meta.graphSample
https://neo4j.com/labs/apoc/4.4/overview/apoc.meta/apoc.meta.cypher.isType
https://neo4j.com/labs/apoc/4.4/overview/apoc.meta/apoc.meta.stats
https://neo4j.com/labs/apoc/4.4/overview/apoc.meta/apoc.meta.data
https://neo4j.com/labs/apoc/4.4/overview/a

In [ ]:
from google.colab import files
import json

j = json.dumps(data)
# open file for writing, "w" 
f = open("/content/drive/MyDrive/neo4j_docs.json","w")

# write json object to file
f.write(j)

# close file
f.close()